### Перетворення типів датасету із рядкового до більш зручного для обробки
_деякі довідкові дані_

In [2]:
import pandas as pd

In [3]:
# 35 sec
# завантажити семпл в df
df = pd.read_csv("/dataset/bd_lab_small_sample.csv")

/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (0,6,8,9,10,11,12,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Перетворити стовпчик `envent_start_date` (тип `str`) на тип   `datetime`
**50 sec**

In [4]:
# timing: 45-50 sec
df.event_start_date = pd.to_datetime(df.event_start_date, format='%d%b%y:%H:%M:%S')

(для довідки) http://strftime.org/ -- що підставляти в параметр `format`

In [5]:
# мінімальна та максимальна дати в датасеті
print(df.event_start_date.min())
print(df.event_start_date.max())

2017-06-01 00:00:00
2017-08-31 23:59:59


#### Перетворення стовчика 'cost' (тип `str`) на  число (`float`)

In [5]:
# 5 sec
df['cost'] = pd.to_numeric(df['cost'],errors='coerce')

#### Групування даних 

In [6]:
# знаходження загальної суми розмов для кожного користувача
df.pivot_table(values = 'cost', index = 'hash_number_A', aggfunc = sum).head(10)

hash_number_A
148752      0.0000
149001    110.9969
149147     51.4993
149207    113.5000
149342      0.0000
149749    177.9030
149854     53.3176
149996    282.2254
150624    282.4945
150964    106.5099
Name: cost, dtype: float64

#### Видалення даних із незаповненою геолокацією

In [10]:
# чи є дані без геолокації (широта 'LAN' == '.')?
import numpy as np
np.any(df['LAT'] == ".")

True

In [11]:
# видаляємо дані, де відсутнє значення довготи LAT = '.'
# записуємо в новий датафрейм df_with_locat
df_with_locat = df[df['LAT'] != '.']

In [12]:
# перевіряємо, чи є ще дані без вказаної широти
np.any(df_with_locat['LAT']==".")

False

In [13]:
# Кількість записів із геолокацією
len(df_with_locat)

10043183

In [14]:
# Для порівняння -- Кількість записів у вихідному датасеті
len(df)

10184607

In [15]:
# чи є довготи LON?
np.any(df_with_locat['LON']==".")

False

#### Кількість людей в радіусі від центру Львова

In [16]:
# Перетворюємо дані широти та довготи із `str` на `float`
df_with_locat.LAT = pd.to_numeric(df_with_locat.LAT, errors = 'coerce')
df_with_locat.LON = pd.to_numeric(df_with_locat.LON, errors = 'coerce')

/opt/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [17]:
# координати центру міста (Львів)
center_lat, center_lon = 49.84, 24.03
# 0.1 градуса в координатах  = 7 км
dist_from_center = 0.2 # 14 км = 7 * 0.2

# виділяємо дані із df_with_locat тільки по Львову
# 
df_Lviv = df_with_locat[(df_with_locat['LAT'] - center_lat)**2 + 
                        (df_with_locat['LON'] - center_lon)**2 < dist_from_center**2]

In [18]:
# дивимося скільки записів залишилося на відстані 0.2 градуси (14 км) від центру Львова
len(df_Lviv)

414062

#### карти -- `mapbox`

In [33]:
import gmplot
import webbrowser

#pathlon = -117.2974695,-117.2980671,-117.2984607,-117.2979182,-117.2974082,-117.2966604,-117.2977518,-117.2987498,-117.2981491,-117.297708,-117.2972804,-117.2965301,-117.2979485,-117.2975028,-117.2980506,-117.2982983,-117.2976609,-117.2970861,-117.2969908,-117.2981731,-117.2987695,-117.2981146,-117.2976909,-117.2969674,-117.2969299,-117.298394
#pathlat = 33.27172039,33.27197757,33.27217535,33.27225324,33.27218351,33.27233921,33.27242614,33.27248971,33.27268346,33.27265944,33.27263664,33.27279608,33.27281652,33.27194103,33.27176546,33.27224514,33.27222714,33.27208829,33.27237357,33.27243373,33.27262189,33.27268296,33.27265933,33.27262125,33.27282274,33.27283925

#pathlat = 49.84, 49.94, 49.94, 49.84, 49.84
#pathlon = 24.03, 24.03, 24.13, 24.13, 24.03


lats = df_Lviv['LAT']
lngs = df_Lviv['LON']

gmap = gmplot.GoogleMapPlotter(49.84, 24.03, 15)
#gmap.scatter(lats, lngs, 'blue', marker=False)
gmap.heatmap(lats, lngs)
#gmap.scatter(lats, lngs, 'k', size = 40, marker=True)

#gmap.plot(pathlat,pathlon,'cornflowerblue', edge_width=10)

gmap.draw('map_Lviv.html')
filename = 'map_Lviv.html'
webbrowser.open_new_tab(filename)

True

In [40]:
#Виділяємо НІЧ
df_Lviv['time_hour']=df_Lviv['event_start_date'].dt.hour

df_Lviv_night = df_Lviv[(df_Lviv['time_hour'] >= 23) | (df_Lviv['time_hour'] <= 5)]


/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [ ]:
import gmplot
import webbrowser

#pathlon = -117.2974695,-117.2980671,-117.2984607,-117.2979182,-117.2974082,-117.2966604,-117.2977518,-117.2987498,-117.2981491,-117.297708,-117.2972804,-117.2965301,-117.2979485,-117.2975028,-117.2980506,-117.2982983,-117.2976609,-117.2970861,-117.2969908,-117.2981731,-117.2987695,-117.2981146,-117.2976909,-117.2969674,-117.2969299,-117.298394
#pathlat = 33.27172039,33.27197757,33.27217535,33.27225324,33.27218351,33.27233921,33.27242614,33.27248971,33.27268346,33.27265944,33.27263664,33.27279608,33.27281652,33.27194103,33.27176546,33.27224514,33.27222714,33.27208829,33.27237357,33.27243373,33.27262189,33.27268296,33.27265933,33.27262125,33.27282274,33.27283925

#pathlat = 49.84, 49.94, 49.94, 49.84, 49.84
#pathlon = 24.03, 24.03, 24.13, 24.13, 24.03


lats = df_Lviv_night['LAT']
lngs = df_Lviv_night['LON']

gmap = gmplot.GoogleMapPlotter(49.84, 24.03, 14)
#gmap.scatter(lats, lngs, 'blue', marker=False)
gmap.heatmap(lats, lngs)
#gmap.scatter(lats, lngs, 'k', size = 40, marker=True)

#gmap.plot(pathlat,pathlon,'cornflowerblue', edge_width=10)

gmap.draw('map_Lviv_night.html')
filename = 'map_Lviv_night.html'
webbrowser.open_new_tab(filename)

In [50]:
# Виділяємо для нічних туристів вихідні
import datetime as dt
df_Lviv_night['weekday'] = df_Lviv_night['event_start_date'].dt.dayofweek

df_567 = df_Lviv_night[(df_Lviv_night.weekday == 5) | (df_Lviv_night.weekday == 6) | (df_Lviv_night.weekday == 7)]


lats = df_567['LAT']
lngs = df_567['LON']

gmap = gmplot.GoogleMapPlotter(49.84, 24.03, 14)
#gmap.scatter(lats, lngs, 'blue', marker=False)
gmap.heatmap(lats, lngs)
#gmap.scatter(lats, lngs, 'k', size = 40, marker=True)

#gmap.plot(pathlat,pathlon,'cornflowerblue', edge_width=10)

gmap.draw('map_Lviv_weekend.html')
filename = 'map_Lviv_weekend.html'
webbrowser.open_new_tab(filename)

/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


True

#### Зворотня геолокація із `geopy`

In [4]:
# Для доустановки geopy в інстансі 
#  sudo /opt/anaconda3/bin/pip install geopy # в PuTTY

In [19]:
from geopy import Nominatim
geolocator = Nominatim()

location = geolocator.reverse("49.84,24.03")
print(location.raw['address']['city'])

try:
    location = geolocator.reverse("48.30,32.16")
    print(location.raw['address']['city'])
except KeyError:
    print('no city here')


Львів
no city here


функція для перевірки входження координати `coord` в місто `city`

In [20]:
from geopy import Nominatim
def isInCity(coord, city):
    """coord -- string with lat, lon coordinates; return True if coord in city, False otherwise"""
    geolocator = Nominatim()
    location = geolocator.reverse(coord)
    try:
        location = geolocator.reverse(coord)
        if location.raw['address']['city'] == city: # "Львів"
            return True
        else: # не "Львів"
            return False
    except KeyError: # немає міста взагалі
        return False


#### Поділ туристи / не туристи кількістю проведених днів
1) якщо більше 14 днів між мінімальною та максимальною датою дзвінка для даного id, коли він у Львові, то "не турист"
2) якщо відсоток дзвінків за розглядуваний період у Львові більше, ніж ____, то "не турист"

In [24]:
# обрахувати час у Львові для кожної людини
time_in_Lviv = df_Lviv.groupby('hash_number_A')['event_start_date'].max() \
- df_Lviv.groupby('hash_number_A')['event_start_date'].min()
time_in_Lviv.head()

hash_number_A
149854    0 days 02:35:32
151558    0 days 00:00:00
151834   37 days 19:37:12
153597   42 days 22:04:53
154194    2 days 19:48:42
Name: event_start_date, dtype: timedelta64[ns]

In [25]:
from datetime import timedelta
dt_tourist = timedelta(days=7)  # граничний час перебування у Львові туриста

# ті, чий час перебування у Львові < 14 днів
tourist_hashes = time_in_Lviv[time_in_Lviv < dt_tourist]
tourists_list = list(tourist_hashes.index)
len(tourist_hashes)

1084

In [26]:
df_Lviv['hash_number_A'].head()

180    2623046
194    2480764
210    2480764
212     619074
218    1077431
Name: hash_number_A, dtype: object

In [ ]:
# всі дзвінки туристів у Львові
#df_Lviv_tourists = df_Lviv[np.in1d(df_Lviv['hash_number_A'], tourists_list)]

df_Lviv_tourists = df_Lviv[np.in1d(df_Lviv['hash_number_A'], tourists_list)]

####  Добавление столбца с номером дня недели

In [23]:
import datetime as dt
df_Lviv_tourists['weekday'] = df_Lviv_tourists['event_start_date'].dt.dayofweek

/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


#### Добавление столбца с временем (час)

In [ ]:
df_Lviv_tourists['time_hour']=df_Lviv_tourists['event_start_date'].dt.hour